In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, lit

spark = SparkSession.builder.appName("Jupyter").getOrCreate()

25/07/26 11:34:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [14]:
events = (spark.read.option("header", "true")
            .csv("/home/iceberg/data/events.csv")
            .withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))    
         )
devices = spark.read.option("header", "true").csv("/home/iceberg/data/devices.csv")
    
df = events.join(devices, on="device_id", how="left").withColumnsRenamed({"browser_type": "browser_family", "os_type": "os_family"})

df.show()

+----------+-----------+--------+--------------------+----------+--------------------+-------------------+--------------+---------+-----------+
| device_id|    user_id|referrer|                host|       url|          event_time|         event_date|browser_family|os_family|device_type|
+----------+-----------+--------+--------------------+----------+--------------------+-------------------+--------------+---------+-----------+
| 532630305| 1037710827|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|2021-03-08 00:00:00|         Other|    Other|      Other|
| 532630305|  925588856|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|2021-05-10 00:00:00|         Other|    Other|      Other|
| 532630305|-1180485268|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|2021-02-17 00:00:00|         Other|    Other|      Other|
| 532630305|-1044833855|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|2021-09-24 00:00:00|         Other|    Other|     

In [22]:
sorted = (df.repartition(10, col("event_date"))
            .sortWithinPartitions(col("event_date"), col("host"), col("browser_family"))
            .withColumn("event_time", col("event_time").cast("timestamp"))
          )
sorted.explain()

sortedTwo = (df.repartition(10, col("event_date"))
            .sort(col("event_date"), col("host"), col("browser_family"))
            .withColumn("event_time", col("event_time").cast("timestamp"))
          )
sortedTwo.explain()


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [device_id#475, user_id#474, referrer#476, host#477, url#478, cast(event_time#479 as timestamp) AS event_time#1021, event_date#486, browser_family#531, os_family#532, device_type#515]
   +- Sort [event_date#486 ASC NULLS FIRST, host#477 ASC NULLS FIRST, browser_family#531 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(event_date#486, 10), REPARTITION_BY_NUM, [plan_id=1417]
         +- Project [device_id#475, user_id#474, referrer#476, host#477, url#478, event_time#479, event_date#486, browser_type#513 AS browser_family#531, os_type#514 AS os_family#532, device_type#515]
            +- BroadcastHashJoin [device_id#475], [device_id#512], LeftOuter, BuildRight, false
               :- Project [user_id#474, device_id#475, referrer#476, host#477, url#478, event_time#479, date_trunc(day, cast(event_time#479 as timestamp), Some(Etc/UTC)) AS event_date#486]
               :  +- FileScan csv [user_id#474,device_id

In [23]:
%%sql
CREATE DATABASE IF NOT EXISTS bootcamp


++
||
++
++

In [24]:
%%sql
DROP TABLE IF EXISTS bootcamp.events

++
||
++
++

In [26]:
%%sql
CREATE TABLE IF NOT EXISTS bootcamp.events (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING ICEBERG
PARTITIONED BY (years(event_date));

++
||
++
++

In [27]:
%%sql
CREATE TABLE IF NOT EXISTS bootcamp.events_sorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING ICEBERG
PARTITIONED BY (years(event_date));

++
||
++
++

In [30]:
%%sql
CREATE TABLE IF NOT EXISTS bootcamp.events_unsorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING ICEBERG
PARTITIONED BY (years(event_date));

++
||
++
++

In [51]:
start_df = (df.repartition(4, col("event_date"))
            .withColumn("event_time", col("event_time").cast("timestamp"))  
            )
first_sort_df = start_df.sortWithinPartitions(col("event_date"), col("browser_family"), col("os_family"))


start_df.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.events_sorted")

In [45]:
%%sql
select * from bootcamp.events_unsorted limit 10;

device_id,user_id,referrer,host,url,event_time,event_date,browser_family,os_family,device_type
532630305,747494706,None,www.zachwilson.tech,/,2021-09-26 16:03:17.535000,2021-09-26 00:00:00,Other,Other,Other
532630305,747494706,None,admin.zachwilson.tech,/,2021-02-21 16:08:17.975000,2021-02-21 00:00:00,Other,Other,Other
532630305,-824540328,None,admin.zachwilson.tech,/,2021-09-28 17:23:14.992000,2021-09-28 00:00:00,Other,Other,Other
-733097781,5890109,None,www.zachwilson.tech,/,2023-01-09 11:55:28.032000,2023-01-09 00:00:00,curl,Other,Other
532630305,-707954698,None,admin.zachwilson.tech,/,2021-06-19 05:55:00.559000,2021-06-19 00:00:00,Other,Other,Other
532630305,747494706,None,admin.zachwilson.tech,/,2021-06-19 15:09:19.519000,2021-06-19 00:00:00,Other,Other,Other
-733097781,5890109,None,www.zachwilson.tech,/,2023-01-09 20:10:27.610000,2023-01-09 00:00:00,curl,Other,Other
532630305,696863716,None,admin.zachwilson.tech,/,2023-01-10 04:43:49.204000,2023-01-10 00:00:00,Other,Other,Other
532630305,-1180485268,None,www.eczachly.com,/,2021-06-28 16:51:11.679000,2021-06-28 00:00:00,Other,Other,Other
532630305,-1180485268,None,www.zachwilson.tech,/,2021-06-28 17:34:44.423000,2021-06-28 00:00:00,Other,Other,Other


In [37]:
%%sql
select * from bootcamp.events_sorted.files;

content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,referenced_data_file,content_offset,content_size_in_bytes,readable_metrics
0,s3://warehouse/bootcamp/events_sorted/data/00000-133-6fe482e3-7684-4d8d-92e5-c51f0046db1c-0-00001.parquet,PARQUET,1,Row(event_date_year=None),89391,1032260,"{1: 107466, 2: 61022, 3: 11455, 4: 12926, 5: 7383, 6: 426449, 7: 2293, 8: 77425, 9: 310063, 10: 10711}","{1: 89391, 2: 89391, 3: 89391, 4: 89391, 5: 89391, 6: 89391, 7: 89391, 8: 89391, 9: 89391, 10: 89391}","{1: 0, 2: 46359, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1, 10: 0}",{},"{1: bytearray(b'/'), 2: bytearray(b'52.20.78.240'), 3: bytearray(b'%E3%82%A6%E3%82%'), 4: bytearray(b'Android'), 5: bytearray(b'aashish.techcrea'), 6: bytearray(b' \xba\xe7\xb8\xa8\xb8\x05\x00'), 7: bytearray(b'\x00\xa0&\xb4\xa8\xb8\x05\x00'), 8: bytearray(b'-100210680'), 9: bytearray(b'-1000095488'), 10: bytearray(b'17MB150WB')}","{1: bytearray(b'/zzageqnf.php?Fp'), 2: bytearray(b'zachwilson.tech'), 3: bytearray(b'webprosbot'), 4: bytearray(b'iOS'), 5: bytearray(b'zachwilson.techd'), 6: bytearray(b'\xe8\xb0\x1b\x8ec\x03\x06\x00'), 7: bytearray(b'\x00\xe0dqO\x03\x06\x00'), 8: bytearray(b'999535123'), 9: bytearray(b'999884938'), 10: bytearray(b'vivo $2')}",None,[4],None,0,None,None,None,"Row(browser_family=Row(column_size=11455, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='%E3%82%A6%E3%82%', upper_bound='webprosbot'), device_id=Row(column_size=77425, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='-100210680', upper_bound='999535123'), device_type=Row(column_size=10711, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='17MB150WB', upper_bound='vivo $2'), event_date=Row(column_size=2293, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2021, 1, 12, 0, 0), upper_bound=datetime.datetime(2023, 8, 20, 0, 0)), event_time=Row(column_size=426449, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2021, 1, 12, 0, 1, 19, 764000), upper_bound=datetime.datetime(2023, 8, 20, 23, 59, 41, 89000)), host=Row(column_size=7383, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='aashish.techcrea', upper_bound='zachwilson.techd'), os_family=Row(column_size=12926, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='Android', upper_bound='iOS'), referrer=Row(column_size=61022, value_count=89391, null_value_count=46359, nan_value_count=None, lower_bound='52.20.78.240', upper_bound='zachwilson.tech'), url=Row(column_size=107466, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='/', upper_bound='/zzageqnf.php?Fp'), user_id=Row(column_size=310063, value_count=89391, null_value_count=1, nan_value_count=None, lower_bound='-1000095488', upper_bound='999884938'))"
0,s3://warehouse/bootcamp/events_sorted/data/00001-134-6fe482e3-7684-4d8d-92e5-c51f0046db1c-0-00001.parquet,PARQUET,1,Row(event_date_year=None),99232,1165545,"{1: 142178, 2: 67363, 3: 11914, 4: 16543, 5: 9119, 6: 475862, 7: 2373, 8: 86514, 9: 337013, 10: 11522}","{1: 99232, 2: 99232, 3: 99232, 4: 99232, 5: 99232, 6: 99232, 7: 99232, 8: 99232, 9: 99232, 10: 99232}","{1: 0, 2: 49299, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 58, 10: 0}",{},"{1: bytearray(b'""/?""""<?=print(93'), 2: bytearray(b'""https://www.goo'), 3: bytearray(b') Bot'), 4: bytearray(b'Android'), 5: bytearray(b'abhishekanand.te'), 6: bytearray(b'(\x83\xb2EX\xb8\x05\x00'), 7: bytearray(b'\x00 \xc9<X\xb8\x05\x00'), 8: bytearray(b'-100210680'), 9: bytearray(b'-1000370060'), 10: bytearray(b'13 Pro Max')}","{1: bytearray(b'/zz.php'), 2: bytearray(b'zachwilson.tech'), 3: bytearray(b'webprosbot'), 4: bytearray(b'iOS'), 5: bytearray(b'zsavi524.techcrf'), 6: bytearray(b'\x88\xb8\x07P;\x03\x06\x00'),

In [52]:
%%sql
select SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted'
from bootcamp.events_sorted.files

UNION ALL

select SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted'
from bootcamp.events_unsorted.files;

size,num_files,sorted
5089149,4,sorted
5556660,4,unsorted
